In [1]:
import os, sys
from functools import cache

import numpy as np
import pandas as pd

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from fetching import fetch_current_season_data
from preprocessing import prep_schedule_data
import soccerdata as sd


[06/13/24 22:36:36] INFO     No custom team name replacements found. You can configure these in       ]8;id=579217;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=526132;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\Marc\soccerdata\config\teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=688343;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=724693;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_config.py#165\165]8;;\
                             C:\Users\Marc\soccerdata\config\league_dict.json.                                     

## Prepartion for strategies

In [2]:
season_data = fetch_current_season_data(use_saved=False, save_path=os.path.join("..", "data"))

Fetching new current season data


                    INFO     Saving cached data to C:\Users\Marc\soccerdata\data\FBref                ]8;id=699615;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=837047;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [3]:
match_days = season_data["week"].max()
table = pd.DataFrame(columns=["Team", "MP", "W", "D", "L", "GF", "GA", "GD", "PTs", "Pts/MP", "xG", "xGA", "xGD", "xGD/90"])
for week in range(1, match_days+1):
    season_data.query("week== @week")

In [4]:
def score_expander(df):
    scores_df = (df['score'].str.split('–', n=1, expand=True)
                 .rename(columns={0:'score_home', 1:'score_away'})
                 .astype({'score_home': int, 'score_away':int}))
    return df.join(scores_df)

season_data_prep = (season_data
    .query("round=='Regular season'")
    .pipe(score_expander)
    .assign(result_home=lambda df: np.where(df["score_home"] > df["score_away"], "W", (np.where(df["score_home"] < df["score_away"], "L", "D"))),
            result_away=lambda df: np.where(df["score_home"] < df["score_away"], "W", (np.where(df["score_home"] > df["score_away"], "L", "D"))),)
)
 

season_data_prep



round  \
league         season game                                                     
GER-Bundesliga 2324   2023-08-18 Werder Bremen-Bayern Munich  Regular season   
                      2023-08-19 Augsburg-Gladbach            Regular season   
                      2023-08-19 Dortmund-Köln                Regular season   
                      2023-08-19 Hoffenheim-Freiburg          Regular season   
                      2023-08-19 Leverkusen-RB Leipzig        Regular season   
...                                                                      ...   
                      2024-05-18 Leverkusen-Augsburg          Regular season   
                      2024-05-18 Stuttgart-Gladbach           Regular season   
                      2024-05-18 Union Berlin-Freiburg        Regular season   
                      2024-05-18 Werder Bremen-Bochum         Regular season   
                      2024-05-18 Wolfsburg-Mainz 05           Regular season   

                                                              week  day  \
league         season game                                                
GER-Bundesliga 2324   2023-08-18 Werder Bremen-Bayern Munich     1  Fri   
                      2023-08-19 Augsburg-Gladbach               1  Sat   
                      2023-08-19 Dortmund-Köln                   1  Sat   
                      2023-08-19 Hoffenheim-Freiburg             1  Sat   
                      2023-08-19 Leverkusen-RB Leipzig           1  Sat   
...                                                            ...  ...   
                      2024-05-18 Leverkusen-Augsburg            34  Sat   
                      2024-05-18 Stuttgart-Gladbach             34  Sat   
                      2024-05-18 Union Berlin-Freiburg          34  Sat   
                      2024-05-18 Werder Bremen-Bochum           34  Sat   
                      2024-05-18 Wolfsburg-Mainz 05             34  Sat   

                                                                   date  \
league         season game                                                
GER-Bundesliga 2324   2023-08-18 Werder Bremen-Bayern Munich 2023-08-18   
                      2023-08-19 Augsburg-Gladbach           2023-08-19   
                      2023-08-19 Dortmund-Köln               2023-08-19   
                      2023-08-19 Hoffenheim-Freiburg         2023-08-19   
                      2023-08-19 Leverkusen-RB Leipzig       2023-08-19   
...                                                                 ...   
                      2024-05-18 Leverkusen-Augsburg         2024-05-18   
                      2024-05-18 Stuttgart-Gladbach          2024-05-18   
                      2024-05-18 Union Berlin-Freiburg       2024-05-18   
                      2024-05-18 Werder Bremen-Bochum        2024-05-18   
                      2024-05-18 Wolfsburg-Mainz 05          2024-05-18   

                                                               time  \
league         season game                                            
GER-Bundesliga 2324   2023-08-18 Werder Bremen-Bayern Munich  20:30   
                      2023-08-19 Augsburg-Gladbach            15:30   
                      2023-08-19 Dortmund-Köln                18:30   
                      2023-08-19 Hoffenheim-Freiburg          15:30   
                      2023-08-19 Leverkusen-RB Leipzig        15:30   
...                                                             ...   
                      2024-05-18 Leverkusen-Augsburg          15:30   
                      2024-05-18 Stuttgart-Gladbach           15:30   
                      2024-05-18 Union Berlin-Freiburg        15:30   
                      2024-05-18 Werder Bremen-Bochum         15:30   
                      2024-05-18 Wolfsburg-Mainz 05           15:30   

                                                                  home_team  \
league         season game                                                 

In [5]:
table = pd.DataFrame(0, columns=["MP", "W", "D", "L", "GF", "GA", "GD", "PTs", "Pts/MP", "xG", "xGA", "xGD", "xGD/90"], index=season_data.query("round == 'Regular season'")["home_team"].unique())
table.index.name = "Team"
table

,MP,W,D,L,GF,GA,GD,PTs,Pts/MP,xG,xGA,xGD,xGD/90
Team,,,,,,,,,,,,,
Werder Bremen,0,0,0,0,0,0,0,0,0,0,0,0,0
Augsburg,0,0,0,0,0,0,0,0,0,0,0,0,0
Dortmund,0,0,0,0,0,0,0,0,0,0,0,0,0
Hoffenheim,0,0,0,0,0,0,0,0,0,0,0,0,0
Leverkusen,0,0,0,0,0,0,0,0,0,0,0,0,0
Stuttgart,0,0,0,0,0,0,0,0,0,0,0,0,0
Wolfsburg,0,0,0,0,0,0,0,0,0,0,0,0,0
Eint Frankfurt,0,0,0,0,0,0,0,0,0,0,0,0,0
Union Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
table = pd.DataFrame(0, columns=["Rk", "MP", "W", "D", "L", "GF", "GA", "GD", "Pts", "Pts/MP", "xG", "xGA", "xGD", "xGD/90"], index=season_data.query("round == 'Regular season'")["home_team"].unique())
table = table.astype({"Pts/MP":float, "xG":float, "xGA": float, "xGD":float, "xGD/90":float})
table.index.name = "Team"
# might need home and away table as well

def process_match(s, table, type="Both"):
    assert type in ("Home", "Away", "Both"), "Type must be one of Home, Away, or Both."
    if type in ("Home", "Both"):
        home_team = s["home_team"]
        stats_before = table.loc[home_team]
        stats_before["MP"] += 1
        stats_before[s["result_home"]] += 1
        stats_before["GF"] += s["score_home"]
        stats_before["GA"] += s["score_away"]
        stats_before["GD"] = stats_before["GF"] - stats_before["GA"]
        stats_before["Pts"] += 3 if s["result_home"] == "W" else (1 if s["result_home"] == "D" else 0)
        stats_before["Pts/MP"] = stats_before["Pts"] / stats_before["MP"]
        stats_before["xG"] += s["home_xg"]
        stats_before["xGA"] += s["away_xg"]
        stats_before["xGD"] = stats_before["xG"] - stats_before["xGA"]
        stats_before["xGD/90"] = stats_before["xGD"] / stats_before["MP"]
        
        table.loc[home_team] = stats_before
    
    
    if type in ("Away", "Both"):
        away_team = s["away_team"]
        stats_before = table.loc[away_team]
        stats_before["MP"] += 1
        stats_before[s["result_away"]] += 1
        stats_before["GF"] += s["score_away"]
        stats_before["GA"] += s["score_home"]
        stats_before["GD"] = stats_before["GF"] - stats_before["GA"]
        stats_before["Pts"] += 3 if s["result_away"] == "W" else (1 if s["result_away"] == "D" else 0)
        stats_before["Pts/MP"] = stats_before["Pts"] / stats_before["MP"]
        stats_before["xG"] += s["away_xg"]
        stats_before["xGA"] += s["home_xg"]
        stats_before["xGD"] = stats_before["xG"] - stats_before["xGA"]
        stats_before["xGD/90"] = stats_before["xGD"] / stats_before["MP"]
        
        table.loc[away_team] = stats_before 

tables = {}
for week in range(1, season_data_prep["week"].max() +1):
    season_data_prep.query("week == @week").apply(process_match, args=(table,"Both"), axis=1)
    table = (table
            .sort_values(by=["Pts", "GD", "GF"], ascending=[False, False, False])
            .assign(Rk=lambda df: range(1,df.shape[0]+1))
            )
    tables[week] = table.copy(deep=True)



# Update table(s)
# season_data.query("week== 1")["match_report"] for xG and other stats
# season_data_prep
table

,Rk,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90
Team,,,,,,,,,,,,,,
Leverkusen,1,34,28,6,0,89,24,65,90,2.647059,73.5,29.9,43.6,1.282353
Stuttgart,2,34,23,4,7,78,39,39,73,2.147059,70.8,37.9,32.9,0.967647
Bayern Munich,3,34,23,3,8,94,45,49,72,2.117647,85.3,32.2,53.1,1.561765
RB Leipzig,4,34,19,8,7,77,39,38,65,1.911765,65.5,39.9,25.6,0.752941
Dortmund,5,34,18,9,7,68,43,25,63,1.852941,63.8,51.8,12.0,0.352941
Eint Frankfurt,6,34,11,14,9,51,50,1,47,1.382353,45.8,52.0,-6.2,-0.182353
Hoffenheim,7,34,13,7,14,66,66,0,46,1.352941,53.7,60.3,-6.6,-0.194118
Heidenheim,8,34,10,12,12,50,55,-5,42,1.235294,39.1,62.0,-22.9,-0.673529
Werder Bremen,9,34,11,9,14,48,54,-6,42,1.235294,46.8,56.6,-9.8,-0.288235


In [7]:


def score_expander(df):
    scores_df = (df['score'].str.split('–', n=1, expand=True)
                 .rename(columns={0:'score_home', 1:'score_away'})
                 .astype({'score_home': int, 'score_away':int}))
    return df.join(scores_df)

def determine_points(s):
    if s["result"] != s["prediction_result"]:
        return 0
    elif (s["score_home"] == s["prediction_home"]) and (s["score_away"] == s["prediction_away"]):
        return 4
    elif (s["score_home"] - s["score_away"]) == (s["prediction_home"]-s["prediction_away"]):
        return 3
    else: 
        return 2

# np.where(df["result"] != df["prediction_result"], 0, (np.where((df["score_home"] == df["prediction_home"]) & (df["score_away"] == df["prediction_away"]), 4, 2)))

# season_data_prep = (season_data
#     .query("round=='Regular season'")
#     .pipe(score_expander)
# )
 
def apply_strategy(df, strategy): 
    df_points = df.assign(prediction_home=lambda df: strategy(df), 
            prediction_away=lambda df: strategy(df, home=False),
            result=lambda df: np.where(df["score_home"] > df["score_away"], "W", (np.where(df["score_home"] < df["score_away"], "L", "D"))),
            prediction_result= lambda df: np.where(df["prediction_home"] > df["prediction_away"], "W", (np.where(df["prediction_home"] < df["prediction_away"], "L", "D"))),
            points=lambda df: df.apply(determine_points, axis=1),
    )
    return df_points["points"].sum()




## Strategy evaluation

In [8]:
def simple_two_one(df, home=True):
    if home:
        return 2
    else:
        return 1
    
def simple_one_zero(df, home=True):
    if home:
        return 1
    else:
        return 0
    
def simple_draw(df, home=True):
    if home:
        return 0
    else:
        return 0   
    
def simple_draw_1(df, home=True):
    if home:
        return 1
    else:
        return 1   
    
def simple_one_two(df, home=True):
    if home:
        return 1
    else:
        return 2
    
def simple_zero_one(df, home=True):
    if home:
        return 0
    else:
        return 1
    
points = apply_strategy(season_data_prep, strategy=simple_two_one)
print(f"Predicting 2-1 gave {points} points")

points = apply_strategy(season_data_prep, strategy=simple_one_zero)
print(f"Predicting 1-0 gave {points} points")

points = apply_strategy(season_data_prep, strategy=simple_draw)
print(f"Predicting 0-0 gave {points} points")

points = apply_strategy(season_data_prep, strategy=simple_draw_1)
print(f"Predicting 1-1 gave {points} points")

points = apply_strategy(season_data_prep, strategy=simple_one_two)
print(f"Predicting 1-2 gave {points} points")

points = apply_strategy(season_data_prep, strategy=simple_zero_one)
print(f"Predicting 0-1 gave {points} points")

Predicting 2-1 gave 338 points
Predicting 1-0 gave 329 points
Predicting 0-0 gave 256 points
Predicting 1-1 gave 284 points
Predicting 1-2 gave 247 points
Predicting 0-1 gave 247 points


In [24]:
# immer nach aktueller Tabelle tippen
def adaptive_two_zero(s, home=True):
    week = s["week"]
    goals = 0
    if week == 1:
        goals = 2 if home else 1
    else:
        current_table = tables[week-1]
        home_rk = current_table.loc[s["home_team"], "Rk"]
        away_rk = current_table.loc[s["away_team"], "Rk"]
        if home_rk < away_rk:
            goals = 2 if home else 1
        else:
            goals = 1 if home else 2

    return goals
    

def apply_adaptive_strategy(df, strategy): 
    
    df_points = df.assign(prediction_home=lambda df: df.apply(strategy, args=(True,), axis=1), 
            prediction_away=lambda df: df.apply(strategy, args=(False,), axis=1),
            result=lambda df: np.where(df["score_home"] > df["score_away"], "W", (np.where(df["score_home"] < df["score_away"], "L", "D"))),
            prediction_result= lambda df: np.where(df["prediction_home"] > df["prediction_away"], "W", (np.where(df["prediction_home"] < df["prediction_away"], "L", "D"))),
            points=lambda df: df.apply(determine_points, axis=1),
    )
    # display(df_points)
    return df_points["points"].sum()

points = apply_adaptive_strategy(season_data_prep, strategy=adaptive_two_zero)
print(f"Adaptive 2-1 gave {points} points")

Adaptive 2-1 gave 393 points


## Build Model

In [3]:
# Download all historic data first
DATA_PATH = os.path.join("..", "data", "all")

def _get_current_season():
    current_year = pd.Timestamp.today().year
    current_month = pd.Timestamp.today().month
    if current_month > 6 :
        return current_year 
    else:
        return current_year - 1 

def fetch_season_data(season, save_path="data"):
    filename = f"{season}_schedule.csv"
    filepath = os.path.join(save_path, filename)
    
    if os.path.isfile(filepath):
        return pd.read_csv(filepath)
    else:
        fbref = sd.FBref(leagues="GER-Bundesliga", seasons=str(season)[2:])
        schedule = fbref.read_schedule()
        schedule.to_csv(filepath, index=False)
        return schedule
    
def load_season_data(save_path="data"):   
    current_season = _get_current_season()
    while current_season >= 1988:
        print(f"Fetching season {current_season}")
        _ = fetch_season_data(current_season, save_path=save_path)
        current_season -= 1
    
load_season_data(save_path=DATA_PATH)


Fetching season 2023
Fetching season 2022
Fetching season 2021
Fetching season 2020
Fetching season 2019
Fetching season 2018
Fetching season 2017
Fetching season 2016
Fetching season 2015
Fetching season 2014
Fetching season 2013
Fetching season 2012
Fetching season 2011
Fetching season 2010
Fetching season 2009
Fetching season 2008
Fetching season 2007
Fetching season 2006
Fetching season 2005
Fetching season 2004
Fetching season 2003
Fetching season 2002
Fetching season 2001
Fetching season 2000
Fetching season 1999
Fetching season 1998
Fetching season 1997
Fetching season 1996
Fetching season 1995
Fetching season 1994
Fetching season 1993
Fetching season 1992
Fetching season 1991
Fetching season 1990
Fetching season 1989
Fetching season 1988


In [15]:
clubs = pd.read_html("https://fbref.com/en/country/clubs/GER/Germany-Football-Clubs", extract_links="body")

In [34]:
{name: link for name, link in clubs[0].explode("Gender").query("Gender == 'M'")["Squad"].to_list() if "gladbach" in name }

{'1. FC Mönchengladbach U17': '/en/squads/89b05c5a/history/FC-Monchengladbach-U17-Stats-and-History',
 '1. FC Mönchengladbach U19': '/en/squads/d127e5eb/history/FC-Monchengladbach-U19-Stats-and-History',
 'Borussia Mönchengladbach': '/en/squads/32f3ee20/history/Monchengladbach-Stats-and-History',
 'Borussia Mönchengladbach U17': '/en/squads/50eaaf29/history/Monchengladbach-U17-Stats-and-History',
 'Borussia Mönchengladbach U19': '/en/squads/df6f068b/history/Monchengladbach-U19-Stats-and-History'}

In [32]:
{link.split("/")[-1].replace("-Stats-and-History", ""): link.split("/")[-3] for name, link in clubs[0].explode("Gender").query("Gender == 'M'")["Squad"].to_list()}

{'1-CfR-Pforzheim': 'cf1eed21',
 '1-FC-Duren': 'd135e533',
 '1-FC-Germania-EgestorfLangreder': '75515a4a',
 'Heidenheim': '18d9d2a7',
 'Heidenheim-U17': '6569155a',
 'Heidenheim-U19': 'c0cb86fe',
 '1-FC-Kaan-Marienborn-07': '74c507da',
 'Kaiserslautern': '73a27a73',
 'Kaiserslautern-U17': 'f5369b66',
 'Kaiserslautern-U19': 'b58231eb',
 'Koln': 'bc357bf7',
 'Koln-U17': '5b499073',
 'Koln-U19': '7978e9a3',
 '1-FC-Lok-Stendal': '743a93f0',
 'Lokomotive-Leipzig': '473affe3',
 'Magdeburg': 'e18a73da',
 'Magdeburg-U17': '0d59f5f5',
 'Magdeburg-U19': 'ebf5a2a9',
 'FC-Monchengladbach-U17': '89b05c5a',
 'FC-Monchengladbach-U19': 'd127e5eb',
 '1-FC-Neubrandenburg-04': 'dfc25351',
 'Nurnberg': '6f2c108c',
 'Nurnberg-U17': '8724a375',
 'Nurnberg-U19': '6ed99f94',
 'Rielasingen-Arlen': '01f618d1',
 'Saarbrucken': 'eb4b278c',
 'Saarbrucken-U17': 'fe7445dc',
 'Saarbrucken-U19': '13ecb521',
 'Schweinfurt': 'c4375806',
 'Union-Berlin': '7a41008f',
 'Union-Berlin-U17': '50e620c4',
 'Union-Berlin-U19': '

In [ ]:
# How to get club IDs to season match?

In [ ]:
# Kleines modell mit aktueller Tabelle, Statistiken der letzten 5/10 matches (form), h2h

# Für Pokal/CL -> Daten für internationale Teams benötigt -> Einfach Ranglisten (Fifa)

In [15]:
def _get_current_season():
    current_year = pd.Timestamp.today().year
    current_month = pd.Timestamp.today().month
    if current_month > 6 :
        return current_year 
    else:
        return current_year - 1 


def fetch_season_data(season, use_saved=False, save_path="data"):
    if not season:
        current_season = _get_current_season()
    else:
        current_season = season
        
    if not use_saved:
        fbref = sd.FBref(leagues="GER-Bundesliga", seasons=str(current_season)[2:])
        schedule = fbref.read_schedule()
        schedule.to_csv(os.path.join(save_path, f"{current_season}_schedule.csv"), index=False)
    else: 
        schedule = pd.read_csv(os.path.join(save_path, f"{current_season}_schedule.csv"), parse_dates=["date"])
    
    return schedule


def score_expander(df):
    if df.shape[0] > 0:
        scores_df = (df['score'].apply(lambda val: val if len(val.split(" ")) == 1 else val.split(" ")[1])
                      .str.split('–', n=2, expand=True)
                      .rename(columns={0:'score_home', 1:'score_away'})
                      .astype({'score_home': int, 'score_away':int})
                    )
    else:
        scores_df = pd.DataFrame(columns=['score_home', 'score_away'])
    return df.join(scores_df)

def preprocess_season_matches(df):
    

    return (df
        .query("round=='Regular season'")
        .pipe(score_expander)
        .assign(result_home=lambda df: np.where(df["score_home"] > df["score_away"], "W", (np.where(df["score_home"] < df["score_away"], "L", "D"))),
                result_away=lambda df: np.where(df["score_home"] < df["score_away"], "W", (np.where(df["score_home"] > df["score_away"], "L", "D"))),)
    )


def rank_table(table):
    return (table
            .sort_values(by=["Pts", "GD", "GF"], ascending=[False, False, False])
            .assign(Rk=lambda df: range(1,df.shape[0]+1))
            )

def build_table_from_matches(matches_df, teams):
    
    # might need home and away table as well
    
    def empty_table():
        table = pd.DataFrame(0, columns=["Rk", "MP", "W", "D", "L", "GF", "GA", "GD", "Pts", "Pts/MP", "xG", "xGA", "xGD", "xGD/90"], 
                                    index=teams)
        table = table.astype({"Pts/MP":float, "xG":float, "xGA": float, "xGD":float, "xGD/90":float})
        table.index.name = "Team"
        return table
    

    def process_match(s, table, type="Both"):
        assert type in ("Home", "Away", "Both"), "Type must be one of Home, Away, or Both."
        if type in ("Home", "Both"):
            home_team = s["home_team"]
            stats_before = table.loc[home_team]
            stats_before["MP"] += 1
            stats_before[s["result_home"]] += 1
            stats_before["GF"] += s["score_home"]
            stats_before["GA"] += s["score_away"]
            stats_before["GD"] = stats_before["GF"] - stats_before["GA"]
            stats_before["Pts"] += 3 if s["result_home"] == "W" else (1 if s["result_home"] == "D" else 0)
            stats_before["Pts/MP"] = stats_before["Pts"] / stats_before["MP"]
            if s["away_xg"] != np.NAN and s["home_xg"]!= np.NAN:
                stats_before["xG"] += s["home_xg"]
                stats_before["xGA"] += s["away_xg"]
                stats_before["xGD"] = stats_before["xG"] - stats_before["xGA"]
                stats_before["xGD/90"] = stats_before["xGD"] / stats_before["MP"]
            
            table.loc[home_team] = stats_before
        
        
        if type in ("Away", "Both"):
            away_team = s["away_team"]
            stats_before = table.loc[away_team]
            stats_before["MP"] += 1
            stats_before[s["result_away"]] += 1
            stats_before["GF"] += s["score_away"]
            stats_before["GA"] += s["score_home"]
            stats_before["GD"] = stats_before["GF"] - stats_before["GA"]
            stats_before["Pts"] += 3 if s["result_away"] == "W" else (1 if s["result_away"] == "D" else 0)
            stats_before["Pts/MP"] = stats_before["Pts"] / stats_before["MP"]
            if s["away_xg"] != np.NAN and s["home_xg"]!= np.NAN:
                stats_before["xG"] += s["away_xg"]
                stats_before["xGA"] += s["home_xg"]
                stats_before["xGD"] = stats_before["xG"] - stats_before["xGA"]
                stats_before["xGD/90"] = stats_before["xGD"] / stats_before["MP"]
            
            table.loc[away_team] = stats_before 

    

    table_total = empty_table()
    table_home = empty_table()
    table_away = empty_table()

    matches_df.apply(process_match, args=(table_total, "Both"), axis=1)
    matches_df.apply(process_match, args=(table_home, "Home"), axis=1)
    matches_df.apply(process_match, args=(table_away, "Away"), axis=1)
    for table in [table_total, table_home, table_away]:
        table = (table
                .sort_values(by=["Pts", "GD", "GF"], ascending=[False, False, False])
                .assign(Rk=lambda df: range(1,df.shape[0]+1))
                )
    
    table_total = table_total.pipe(rank_table)
    table_home = table_home.pipe(rank_table)
    table_away = table_away.pipe(rank_table)   
    
    return table_total, table_home, table_away

def build_form_table_from_matches(matches_df, teams, num_past=[5,10]):
    
    def build_form_table_row(team, num=5):
        matches =  (matches_df
                        .query("home_team == @team or away_team == @team")
                        .sort_values(by="datetime")
                        .iloc[-num:])

        form_table, _, _ = build_table_from_matches(matches, teams=teams)
        return form_table.query("Team == @team")
        
    form_tables = []
    for num in num_past:
        form_tab = pd.DataFrame()
        for team in teams:
            table_row = build_form_table_row(team, num=num)
            form_tab = pd.concat([form_tab, table_row], axis=0)
        
        form_tab = form_tab.pipe(rank_table)
        form_tables += [form_tab]
    
    
    return form_tables

# Only need this for IDs
def fetch_team_season_data(season, use_saved=False, save_path="data"):
    if not season:
        current_season = _get_current_season()
    else:
        current_season = season

    if not use_saved:
        fbref = sd.FBref(leagues="GER-Bundesliga", seasons=str(current_season)[2:])
        team_season_stats = (fbref.read_team_season_stats()
                                .assign(team_id=lambda df: df[("url", "")].str.split("/").str[3])
                                .reset_index()
                                .drop(columns=["league", "season", "url"], level=0))
        team_season_stats.to_csv(os.path.join(save_path, f"{current_season}_team_stats.csv"), index=False)
    else: 
        team_season_stats = pd.read_csv(os.path.join(save_path, f"{current_season}_team_stats.csv"), header=[0,1])
        fixed_column_tuples = [(colA, '') if colB.startswith("Unnamed:") else (colA, colB)  for colA, colB in team_season_stats.columns]
        team_season_stats.columns = pd.MultiIndex.from_tuples(fixed_column_tuples)
    
    return team_season_stats


# Use caching to save h2h of clubs
@cache
def fetch_head_to_head_data(home_club_id, away_club_id, use_saved=False, save_path="data"):
    print(f"Fetching club season data")
    
    if not use_saved:
        url = f"https://fbref.com/en/stathead/matchup/teams/{home_club_id}/{away_club_id}/"

        h2h_table = (pd.read_html(url)[0]
                     .drop(columns=["Day", "Attendance", "Venue", "Referee", "Match Report", "Notes"])
                     .query("Comp != 'Comp'")
                     .fillna({"Time": "00:00"})
                     .assign(datetime=lambda df: pd.to_datetime(df["Date"].astype(str) + ' ' + df["Time"], utc=True))
                     )
        h2h_table.to_csv(os.path.join(save_path, f"h2h_{home_club_id}_{away_club_id}.csv"))
    else:
        h2h_table = pd.read_csv(os.path.join(save_path, f"h2h_{home_club_id}_{away_club_id}.csv"), parse_dates=["datetime"])
        
    return h2h_table


def compute_h2h_statistics(matches, home_team, away_team):
    df, _, _ = build_table_from_matches(matches, [home_team, away_team])
    
    
    
    return df


In [16]:
# Build dataset
# Season 2324 -> 2023
def build_dataset_for_season(season = 2023):
    season_data = fetch_season_data(season=season, use_saved=True, save_path=os.path.join("..", "data"))
    season_data = (season_data
                   .query("round == 'Regular season'")
                   .assign(datetime=lambda df: pd.to_datetime(df["date"].astype(str) + ' ' + df["time"], utc=True))
                   .pipe(preprocess_season_matches)
                   )

    teams = season_data.query("round == 'Regular season'")["home_team"].unique().tolist()
    
    team_season_data = fetch_team_season_data(season, use_saved=True, save_path=os.path.join("..", "data"))
    team_ids = team_season_data.set_index("team")["team_id"].to_dict()
    if "M'Gladbach" in team_ids:
        team_ids["Gladbach"] = team_ids["M'Gladbach"]

    dataset = pd.DataFrame()
    
    def build_row_dataset(match_row):
        rowdata = match_row
        home_team = match_row["home_team"].iloc[0]
        away_team = match_row["away_team"].iloc[0]
        
        past_matches = (season_data.assign(match_datetime=rowdata["datetime"].iloc[0], 
                                   in_past=lambda df: (df["match_datetime"] - df["datetime"]).dt.total_seconds() > 0)
                            .query("in_past == True"))
        
        row_ds = pd.DataFrame()
        
        # concat tables together
        table_total, table_home, table_away = build_table_from_matches(past_matches, teams)
        for table, suffix in zip([table_total, table_home, table_away], ["total", "home", "away"]):
            a = table.loc[home_team]
            b = table.loc[away_team]
            for s, suffix_2 in zip([table.loc[home_team], table.loc[away_team], a-b], [suffix + '_home', suffix + '_away', suffix + '_diff'] ):
                s_prep = s.to_frame().T.reset_index(drop=True)
                s_prep.columns = [col + "_" + suffix_2 for col in s_prep.columns]
                row_ds = pd.concat([row_ds, s_prep], axis=1)
        
        
        
        # Form tables
        form_tables = build_form_table_from_matches(past_matches, teams=teams, num_past=[5,10])
        
        form_table_5 = form_tables[0]
        form_table_10 = form_tables[1]
        
        # concat tables together
        table_total, table_home, table_away = build_table_from_matches(past_matches, teams)
        for table, suffix in zip([form_table_5, form_table_10], ["form5", "form10"]):
            a = table.loc[home_team]
            b = table.loc[away_team]
            for s, suffix_2 in zip([table.loc[home_team], table.loc[away_team], a-b], [suffix + '_home', suffix + '_away', suffix + '_diff'] ):
                s_prep = s.to_frame().T.reset_index(drop=True)
                s_prep.columns = [col + "_" + suffix_2 for col in s_prep.columns]
                row_ds = pd.concat([row_ds, s_prep], axis=1)
        
        
        # Head-to-head tables
        h2h_data = fetch_head_to_head_data(team_ids[home_team], team_ids[away_team], use_saved=False, save_path=os.path.join("..", "data"))
        display(h2h_data)
        past_h2h_matches = (h2h_data
                            .assign(match_datetime=rowdata["datetime"].iloc[0], 
                                    days_past=lambda df: (df["match_datetime"] - df["datetime"]).dt.total_seconds()/(60 * 60 * 24),
                                    in_past=lambda df: (df["match_datetime"] - df["datetime"]).dt.total_seconds() > 0)
                            .query("in_past == True")
                            .sort_values("datetime", ascending=False)
                            .rename(columns={"Score":"score", "Home": "home_team", "Away": "away_team", "xG": "home_xg", "xG.1": "away_xg"})
                            .astype({"home_xg": float, "away_xg":float})
                            .pipe(score_expander)
                            .assign(result_home=lambda df: np.where(df["score_home"] > df["score_away"], "W", (np.where(df["score_home"] < df["score_away"], "L", "D"))),
                                    result_away=lambda df: np.where(df["score_home"] < df["score_away"], "W", (np.where(df["score_home"] > df["score_away"], "L", "D"))))
                )
        
        col_init = ['MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP', 'xG', 'xGA', 'xGD', 'xGD/90']
        
        h2h_by_matches = []
        for num_matches in [3,5,10]:
            past_matches_subset = past_h2h_matches.iloc[:num_matches]
            h2h_stats = compute_h2h_statistics(past_matches_subset, home_team, away_team)
            suffix = f"_matches_{num_matches}"
            h2h_stats = h2h_stats[col_init]
            h2h_stats.columns = [col + suffix for col in h2h_stats.columns]
            
            h2h_by_matches += [h2h_stats]
            
        h2h_by_years = []    
        for years_passed in [1, 3]:
            days = years_passed * 365
            past_matches_subset = past_h2h_matches.query("days_past < @days")
            h2h_stats = compute_h2h_statistics(past_matches_subset, home_team, away_team)
            suffix = f"_years_{years_passed}"
            h2h_stats = h2h_stats[col_init]
            h2h_stats.columns = [col + suffix for col in h2h_stats.columns]
            
            h2h_by_years += [h2h_stats]
            
        for table in h2h_by_matches + h2h_by_years:
            a = table.loc[home_team]
            b = table.loc[away_team]
            for s, suffix_2 in zip([table.loc[home_team], table.loc[away_team], a-b], ['_home', '_away', '_diff'] ):
                s_prep = s.to_frame().T.reset_index(drop=True)
                s_prep.columns = [col + "_form_" + suffix_2 for col in s_prep.columns]
                row_ds = pd.concat([row_ds, s_prep], axis=1)
        

        # TODO: Team Statistics
        

        return row_ds
        
    for index, row in season_data.iterrows(): 
        print("Handling row", index)
        if index < 19:
            continue
        row_data = build_row_dataset(row.to_frame().T)
        dataset = pd.concat([dataset, row_data], axis=0)
    
    
        
    test_match = season_data.loc[lambda df: df.game_id == '48d916da']
    # test_match_1 = season_data.loc[lambda df: df.game_id == '97218e9f'] # first match
    display(test_match)
    # dataset = build_row_dataset(test_match_1)
    
    return dataset



build_dataset_for_season()

Handling row 0
Handling row 1
Handling row 2
Handling row 3
Handling row 4
Handling row 5
Handling row 6
Handling row 7
Handling row 8
Handling row 9
Handling row 10
Handling row 11
Handling row 12
Handling row 13
Handling row 14
Handling row 15
Handling row 16
Handling row 17
Handling row 18
Handling row 19
Fetching club season data


,Comp,Round,Date,Time,Home,xG,Score,xG.1,Away,datetime
0,Bundesliga,Regular season,2024-02-03,15:30,Bochum,0.9,1–1,1.6,Augsburg,2024-02-03 15:30:00+00:00
1,Bundesliga,Regular season,2023-09-02,15:30,Augsburg,1.9,2–2,1.6,Bochum,2023-09-02 15:30:00+00:00
2,Bundesliga,Regular season,2023-05-13,15:30,Bochum,1.2,3–2,1.8,Augsburg,2023-05-13 15:30:00+00:00
3,Bundesliga,Regular season,2022-11-12,15:30,Augsburg,2.2,0–1,1.1,Bochum,2022-11-12 15:30:00+00:00
4,Bundesliga,Regular season,2022-04-24,15:30,Bochum,1.1,0–2,1.9,Augsburg,2022-04-24 15:30:00+00:00
5,Bundesliga,Regular season,2021-12-04,15:30,Augsburg,2.9,2–3,2.1,Bochum,2021-12-04 15:30:00+00:00
6,DFB-Pokal,Round of 32,2021-10-27,18:30,Bochum,NaN,(5) 2–2 (4),NaN,Augsburg,2021-10-27 18:30:00+00:00


Handling row 20
Fetching club season data


,Comp,Round,Date,Time,Home,xG,Score,xG.1,Away,datetime
0,Bundesliga,Regular season,2024-02-03,15:30,Bayern Munich,2.1,3–1,1.0,Gladbach,2024-02-03 15:30:00+00:00
1,Bundesliga,Regular season,2023-09-02,18:30,Gladbach,0.3,1–2,2.4,Bayern Munich,2023-09-02 18:30:00+00:00
2,Bundesliga,Regular season,2023-02-18,15:30,Gladbach,2.4,3–2,1.1,Bayern Munich,2023-02-18 15:30:00+00:00
3,Bundesliga,Regular season,2022-08-27,18:30,Bayern Munich,2.8,1–1,0.5,Gladbach,2022-08-27 18:30:00+00:00
4,Bundesliga,Regular season,2022-01-07,20:30,Bayern Munich,1.9,1–2,1.0,Gladbach,2022-01-07 20:30:00+00:00
...,...,...,...,...,...,...,...,...,...,...
64,Bundesliga,Regular season,1990-10-06,00:00,Bayern Munich,NaN,4–1,NaN,Gladbach,1990-10-06 00:00:00+00:00
65,Bundesliga,Regular season,1989-12-02,00:00,Bayern Munich,NaN,2–0,NaN,Gladbach,1989-12-02 00:00:00+00:00
66,Bundesliga,Regular season,1989-08-05,00:00,Gladbach,NaN,0–0,NaN,Bayern Munich,1989-08-05 00:00:00+00:00
67,Bundesliga,Regular season,1989-04-01,00:00,Gladbach,NaN,2–1,NaN,Bayern Munich,1989-04-01 00:00:00+00:00


Handling row 21
Fetching club season data


...

,Comp,Round,Date,Time,Home,xG,Score,xG.1,Away,datetime
0,Bundesliga,Regular season,2024-02-18,17:30,Bochum,1.7,3–2,3.3,Bayern Munich,2024-02-18 17:30:00+00:00
1,Bundesliga,Regular season,2023-09-23,15:30,Bayern Munich,4.8,7–0,0.1,Bochum,2023-09-23 15:30:00+00:00
2,Bundesliga,Regular season,2023-02-11,15:30,Bayern Munich,3.7,3–0,0.3,Bochum,2023-02-11 15:30:00+00:00
3,Bundesliga,Regular season,2022-08-21,17:30,Bochum,0.7,0–7,2.9,Bayern Munich,2022-08-21 17:30:00+00:00
4,Bundesliga,Regular season,2022-02-12,15:30,Bochum,1.4,4–2,1.9,Bayern Munich,2022-02-12 15:30:00+00:00
5,Bundesliga,Regular season,2021-09-18,15:30,Bayern Munich,5.6,7–0,0.4,Bochum,2021-09-18 15:30:00+00:00
6,DFB-Pokal,Second round,2019-10-29,20:00,Bochum,NaN,1–2,NaN,Bayern Munich,2019-10-29 20:00:00+00:00
7,DFB-Pokal,Quarter-finals,2016-02-10,20:30,Bochum,NaN,0–3,NaN,Bayern Munich,2016-02-10 20:30:00+00:00
8,Bundesliga,Regular season,2010-05-01,00:00,Bayern Munich,NaN,3–1,NaN,Bochum,2010-05-01 00:00:00+00:00
9,Bundesliga,Regular season,2009-12-12,00:00,Bochum,NaN,1–5,NaN,Bayern Munich,2009-12-12 00:00:00+00:00


Handling row 39
Fetching club season data


,Comp,Round,Date,Time,Home,xG,Score,xG.1,Away,datetime
0,Bundesliga,Regular season,2024-02-17,15:30,Wolfsburg,1.8,1–1,1.4,Dortmund,2024-02-17 15:30:00+00:00
1,Bundesliga,Regular season,2023-09-23,15:30,Dortmund,1.7,1–0,0.5,Wolfsburg,2023-09-23 15:30:00+00:00
2,Bundesliga,Regular season,2023-05-07,17:30,Dortmund,4.2,6–0,1.2,Wolfsburg,2023-05-07 17:30:00+00:00
3,Bundesliga,Regular season,2022-11-08,18:30,Wolfsburg,2.4,2–0,1.6,Dortmund,2022-11-08 18:30:00+00:00
4,Bundesliga,Regular season,2022-04-16,15:30,Dortmund,3.0,6–1,1.5,Wolfsburg,2022-04-16 15:30:00+00:00
5,Bundesliga,Regular season,2021-11-27,15:30,Wolfsburg,1.9,1–3,2.3,Dortmund,2021-11-27 15:30:00+00:00
6,Bundesliga,Regular season,2021-04-24,15:30,Wolfsburg,1.2,0–2,1.4,Dortmund,2021-04-24 15:30:00+00:00
7,Bundesliga,Regular season,2021-01-03,15:30,Dortmund,1.8,2–0,2.1,Wolfsburg,2021-01-03 15:30:00+00:00
8,Bundesliga,Regular season,2020-05-23,15:30,Wolfsburg,0.6,0–2,1.0,Dortmund,2020-05-23 15:30:00+00:00
9,Bundesliga,Regular season,2019-11-02,15:30,Dortmund,2.2,3–0,1.5,Wolfsburg,2019-11-02 15:30:00+00:00


Handling row 40
Fetching club season data


HTTPError: HTTP Error 429: Too Many Requests

In [41]:
a = pd.Series({"A": 1, "B":2})
b = pd.Series({"B":-2, "A": 3 })
a-b


A   -2
B    4
dtype: int64

In [66]:
fbref = sd.FBref(leagues="GER-Bundesliga", seasons=23)
team_season_stats = fbref.read_team_season_stats()

[06/08/24 23:17:25] INFO     Saving cached data to C:\Users\Marc\soccerdata\data\FBref                ]8;id=912952;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=453764;file://c:\Users\Marc\.venv\bundesliga_preview\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [67]:
team_season_stats

players_used   Age  Poss Playing Time  \
                                                                        MP   
league         season team                                                   
GER-Bundesliga 2324   Augsburg                 17  25.4  47.5            2   
                      Bayern Munich            18  27.3  66.5            2   
                      Bochum                   19  29.6  38.0            2   
                      Darmstadt 98             19  26.2  56.0            2   
                      Dortmund                 19  27.5  65.0            2   
                      Eint Frankfurt           20  26.8  61.0            2   
                      Freiburg                 19  27.5  49.5            2   
                      Heidenheim               18  26.7  45.0            2   
                      Hoffenheim               19  29.0  56.0            2   
                      Köln                     17  26.6  42.0            2   
                      Leverkusen               17  26.3  52.0            2   
                      M'Gladbach               17  25.7  42.0            2   
                      Mainz 05                 20  26.9  46.0            2   
                      RB Leipzig               19  26.6  55.0            2   
                      Stuttgart                18  25.4  53.0            2   
                      Union Berlin             18  27.9  37.0            2   
                      Werder Bremen            18  28.5  39.5            2   
                      Wolfsburg                17  25.6  49.0            2   

                                                     Performance          ...  \
                                     Starts  Min 90s         Gls Ast G+A  ...   
league         season team                                                ...   
GER-Bundesliga 2324   Augsburg           22  180   2           5   4   9  ...   
                      Bayern Munich      22  180   2           6   5  11  ...   
                      Bochum             22  180   2           1   1   2  ...   
                      Darmstadt 98       22  180   2           1   0   1  ...   
                      Dortmund           22  180   2           2   2   4  ...   
                      Eint Frankfurt     22  180   2           2   2   4  ...   
                      Freiburg           22  180   2           2   1   3  ...   
                      Heidenheim         22  180   2           2   1   3  ...   
                      Hoffenheim         22  180   2           4   1   5  ...   
                      Köln               22  180   2           1   1   2  ...   
                      Leverkusen         22  180   2           6   6  12  ...   
                      M'Gladbach         22  180   2           4   3   7  ...   
                      Mainz 05           22  180   2           2   0   2  ...   
                      RB Leipzig         22  180   2           7   5  12  ...   
                      Stuttgart          22  180   2           6   6  12  ...   
                      Union Berlin       22  180   2           8   7  15  ...   
                      Werder Bremen      22  180   2           0   0   0  ...   
                      Wolfsburg          22  180   2           4   3   7  ...   

                                     Per 90 Minutes                         \
                                                Ast  G+A G-PK G+A-PK    xG   
league         season team                                                   
GER-Bundesliga 2324   Augsburg                  2.0  4.5  2.0    4.0  1.47   
                      Bayern Munich             2.5  5.5  2.5    5.0  2.82   
                      Bochum                    0.5  1.0  0.5    1.0  0.91   
                      Darmstadt 98              0.0  0.5  0.5    0.5   1.4   
                      Dortmund                  1.0  2.0  1.0    2.0   1.5   
                      Eint Frankfurt            1.0  2.0  1.0    2.0  1.

In [64]:
season_data

,round,week,day,date,time,home_team,home_xg,score,away_xg,away_team,attendance,venue,referee,match_report,notes,game_id
0,Regular season,1.0,Fri,2023-08-18,20:30,Werder Bremen,0.6,0–4,2.8,Bayern Munich,42100,Wohninvest Weserstadion,Felix Zwayer,/en/matches/97218e9f/Werder-Bremen-Bayern-Muni...,NaN,97218e9f
1,Regular season,1.0,Sat,2023-08-19,15:30,Augsburg,2.3,4–4,1.7,Gladbach,30660,WWK Arena,Daniel Schlager,/en/matches/aac55b7f/Augsburg-Monchengladbach-...,NaN,aac55b7f
2,Regular season,1.0,Sat,2023-08-19,18:30,Dortmund,1.8,1–0,1.2,Köln,81365,Signal Iduna Park,Deniz Aytekin,/en/matches/c05c14ef/Dortmund-Koln-August-19-2...,NaN,c05c14ef
3,Regular season,1.0,Sat,2023-08-19,15:30,Hoffenheim,1.4,1–2,2.6,Freiburg,25890,PreZero Arena,Sven Jablonski,/en/matches/d1a443e5/Hoffenheim-Freiburg-Augus...,NaN,d1a443e5
4,Regular season,1.0,Sat,2023-08-19,15:30,Leverkusen,1.4,3–2,1.6,RB Leipzig,29464,BayArena,Felix Brych,/en/matches/063d7b6b/Bayer-Leverkusen-RB-Leipz...,NaN,063d7b6b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Regular season,34.0,Sat,2024-05-18,15:30,Union Berlin,2.3,2–1,1.0,Freiburg,22012,Stadion An der Alten Försterei,Christian Dingert,/en/matches/0a7a794b/Union-Berlin-Freiburg-May...,NaN,0a7a794b
304,Regular season,34.0,Sat,2024-05-18,15:30,Werder Bremen,3.0,4–1,2.1,Bochum,42100,Wohninvest Weserstadion,Martin Petersen,/en/matches/0c2b0305/Werder-Bremen-Bochum-May-...,NaN,0c2b0305
305,Regular season,34.0,Sat,2024-05-18,15:30,Wolfsburg,0.8,1–3,1.8,Mainz 05,26627,Volkswagen Arena,Frank Willenborg,/en/matches/31be20b7/Wolfsburg-Mainz-05-May-18...,NaN,31be20b7
306,German 1/2 Relegation/Promotion Play-offs,NaN,Thu,2024-05-23,20:30,Bochum,NaN,0–3,NaN,Düsseldorf,26000,Vonovia Ruhrstadion,NaN,/en/matches/9156fa34/Bochum-Dusseldorf-May-23-...,Leg 1 of 2,9156fa34


## Kicktip Records 

In [35]:
points_2324 = """1. Runde	54
1. Spieltag	16
2. Spieltag	14
3. Spieltag	4
4. Spieltag	7
1. Spieltag - CL	17
5. Spieltag	12
6. Spieltag	16
2. Spieltag - CL	25
7. Spieltag	20
8. Spieltag	19
3. Spieltag - CL	30
9. Spieltag	13
2. Runde	21
10. Spieltag	9
4. Spieltag - CL	28
11. Spieltag	10
12. Spieltag	17
5. Spieltag - CL	30
13. Spieltag	6
Achtelfinale	15
14. Spieltag	0
6. Spieltag - CL	12
15. Spieltag	13
16. Spieltag	8
17. Spieltag	14
18. Spieltag	6
19. Spieltag	10
20. Spieltag	15
Viertelfinale	4
21. Spieltag	8
22. Spieltag	15
Achtelfinale Hin	16
23. Spieltag	9
24. Spieltag	14
25. Spieltag	23
Achtelfinale Rück	16
26. Spieltag	9
27. Spieltag	5
Halbfinale	2
28. Spieltag	11
Viertelfinale Hin	6
29. Spieltag	17
Viertelfinale Rück	3
30. Spieltag	14
31. Spieltag	8
Halbfinale Hin	3
32. Spieltag	9
Halbfinale Rück	0
33. Spieltag	13
34. Spieltag	10
Finale - DFB	2"""

points_2223 = """
1. Spieltag	13
2. Spieltag	11
1. Runde	59
3. Spieltag	6
4. Spieltag	11
5. Spieltag	18
1. Spieltag - CL	27
6. Spieltag	8
2. Spieltag - CL	22
7. Spieltag	8
8. Spieltag	9
3. Spieltag - CL	16
9. Spieltag	8
4. Spieltag - CL	20
10. Spieltag	15
2. Runde	31
11. Spieltag	10
5. Spieltag - CL	20
12. Spieltag	15
6. Spieltag - CL	28
13. Spieltag	15
14. Spieltag	8
15. Spieltag	14
16. Spieltag	10
17. Spieltag	15
18. Spieltag	10
Achtelfinale	18
19. Spieltag	8
20. Spieltag	14
Achtelfinale Hin	19
21. Spieltag	10
22. Spieltag	10
23. Spieltag	8
Achtelfinale Rück	8
24. Spieltag	9
25. Spieltag	6
26. Spieltag	6
Viertelfinale	4
27. Spieltag	8
Viertelfinale Hin	5
28. Spieltag	5
Viertelfinale Rück	2
29. Spieltag	14
30. Spieltag	5
Halbfinale	4
31. Spieltag	4
Halbfinale Hin	0
32. Spieltag	12
Halbfinale Rück	3
33. Spieltag	10
34. Spieltag	18
Finale	2
Finale - CL	2"""

points_2122 = """1. Runde	63
1. Spieltag	14
2. Spieltag	7
3. Spieltag	21
4. Spieltag	14
1. Spieltag - CL	16
5. Spieltag	14
6. Spieltag	10
2. Spieltag - CL	17
7. Spieltag	13
8. Spieltag	10
3. Spieltag - CL	27
9. Spieltag	10
2. Runde	23
10. Spieltag	14
4. Spieltag - CL	23
11. Spieltag	7
12. Spieltag	8
5. Spieltag - CL	25
13. Spieltag	12
14. Spieltag	9
6. Spieltag - CL	19
15. Spieltag	8
16. Spieltag	11
17. Spieltag	10
18. Spieltag	10
19. Spieltag	19
Achtelfinale	5
20. Spieltag	15
21. Spieltag	21
22. Spieltag	10
Achtelfinale Hin	17
23. Spieltag	18
24. Spieltag	14
Viertelfinale	12
25. Spieltag	4
Achtelfinale Rück	6
26. Spieltag	12
27. Spieltag	10
28. Spieltag	4
Viertelfinale Hin	6
29. Spieltag	17
Viertelfinale Rück	0
30. Spieltag	11
Halbfinale	5
31. Spieltag	7
Halbfinale Hin	5
32. Spieltag	8
Halbfinale Rück	5
33. Spieltag	13
34. Spieltag	12
Finale	2
Finale - CL	0"""

def prep_points(points_str):
    liga = {row[0]: int(row[1]) for row in [row.split("\t") for row in points_str.split("\n") if "Spieltag" in row.split("\t")[0] and "CL" not in row.split("\t")[0]]}
    pokal = {row[0]: int(row[1]) for row in [row.split("\t") for row in points_str.split("\n") if "Runde" in row.split("\t")[0] or row.split("\t")[0] in ("Achtelfinale", "Viertelfinale", "Halbfinale", "Finale - DFB")]}
    cl = {row[0]: int(row[1]) for row in [row.split("\t") for row in points_str.split("\n") if "CL" in row.split("\t")[0] or "Hin" in row.split("\t")[0] or "Rück" in row.split("\t")[0] ]}
    
    def dict_to_df (input):
        return pd.DataFrame({"Spieltag": input.keys(), "points": input.values()})
    
    return dict_to_df(liga), dict_to_df(pokal), dict_to_df(cl)

liga_2324, pokal_2324, cl_2324 = prep_points(points_2324)
liga_2223, pokal_2223, cl_2223 = prep_points(points_2223)
liga_2122, pokal_2122, cl_2122 = prep_points(points_2122)


In [39]:
liga_2324["points"].sum()

394